In [1]:
from langchain_community.utilities import SQLDatabase

db_file = "marketing_test.db"
 
db = SQLDatabase.from_uri(f"sqlite:///{db_file}")

In [2]:
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect="SQLite",
    top_k=5,
)

In [3]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from src.ai.llm.azure_client import AzureChatModel

toolkit = SQLDatabaseToolkit(db=db, llm=AzureChatModel().get_client())

tools = toolkit.get_tools()

agent_executor = create_react_agent(AzureChatModel().get_client(), tools, prompt=system_message)

In [20]:
question = "Có bao nhiêu chiến dịch truyền thông trên tiktok?"

output_state = agent_executor.invoke({"messages": [{"role": "user", "content": question}]})



================================ Human Message =================================

Có bao nhiêu chiến dịch truyền thông trên tiktok?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_qyRMRRsuM5a37hXQqp2OoRoA)
 Call ID: call_qyRMRRsuM5a37hXQqp2OoRoA
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

AdSpend, Campaign, Customers, Feedback, Leads, Sales
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_RXOLVNJS7ByPwjKEaInycNqY)
 Call ID: call_RXOLVNJS7ByPwjKEaInycNqY
  Args:
    table_names: Campaign
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Campaign" (
	"CampaignID" INTEGER, 
	"Name" TEXT, 
	"Channel" TEXT, 
	"StartDate" DATE, 
	"EndDate" DATE, 
	"Budget" REAL, 
	PRIMARY KEY ("CampaignID")
)

/*
3 rows from Campaign tab

In [13]:
db.run("SELECT * FROM CAMPAIGN WHERE CHANNEL = 'TikTok'")

"[(2, 'Campaign 2', 'TikTok', '2025-03-28', '2025-05-11', 4123.0)]"

In [27]:
for step in output_state["messages"]:
    print(type(step))
    print()
    print(step.content)
    print()

<class 'langchain_core.messages.human.HumanMessage'>

Có bao nhiêu chiến dịch truyền thông trên tiktok?

<class 'langchain_core.messages.ai.AIMessage'>



<class 'langchain_core.messages.tool.ToolMessage'>

AdSpend, Campaign, Customers, Feedback, Leads, Sales

<class 'langchain_core.messages.ai.AIMessage'>



<class 'langchain_core.messages.tool.ToolMessage'>


CREATE TABLE "Campaign" (
	"CampaignID" INTEGER, 
	"Name" TEXT, 
	"Channel" TEXT, 
	"StartDate" DATE, 
	"EndDate" DATE, 
	"Budget" REAL, 
	PRIMARY KEY ("CampaignID")
)

/*
3 rows from Campaign table:
CampaignID	Name	Channel	StartDate	EndDate	Budget
1	Campaign 1	Email	2024-11-29	2024-12-17	3109.8
2	Campaign 2	TikTok	2025-03-28	2025-05-11	4123.0
3	Campaign 3	Facebook	2025-01-31	2025-03-11	1748.71
*/

<class 'langchain_core.messages.ai.AIMessage'>



<class 'langchain_core.messages.tool.ToolMessage'>

SELECT COUNT(*) AS TikTokCampaignCount FROM Campaign WHERE Channel = 'TikTok';

<class 'langchain_core.messages.ai.AIMessage'>



<c

In [35]:
output_state["messages"][6]

ToolMessage(content="SELECT COUNT(*) AS TikTokCampaignCount FROM Campaign WHERE Channel = 'TikTok';", name='sql_db_query_checker', id='8b3cea55-bb89-498d-bd28-b824166134fb', tool_call_id='call_Q9oui9Aq5wyFS5LvvOuVlJ8q')

In [36]:
output_state

{'messages': [HumanMessage(content='Có bao nhiêu chiến dịch truyền thông trên tiktok?', additional_kwargs={}, response_metadata={}, id='b7644129-593e-4bb8-b033-85c0c420687f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qyRMRRsuM5a37hXQqp2OoRoA', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 530, 'total_tokens': 543, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4f3d32ad4e', 'id': 'chatcmpl-CGMZ98Y9pEstm8cq2Ypja7kJ8eAMw', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

HumanMessage.pretty_print

(output_state["messages"][0])

langchain_core.messages.human.HumanMessage

In [48]:
print(HumanMessage(content="Hello").pretty_repr())

================================ Human Message =================================

Hello


In [45]:
for message in output_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Có bao nhiêu chiến dịch truyền thông trên tiktok?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_qyRMRRsuM5a37hXQqp2OoRoA)
 Call ID: call_qyRMRRsuM5a37hXQqp2OoRoA
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

AdSpend, Campaign, Customers, Feedback, Leads, Sales
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_RXOLVNJS7ByPwjKEaInycNqY)
 Call ID: call_RXOLVNJS7ByPwjKEaInycNqY
  Args:
    table_names: Campaign
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Campaign" (
	"CampaignID" INTEGER, 
	"Name" TEXT, 
	"Channel" TEXT, 
	"StartDate" DATE, 
	"EndDate" DATE, 
	"Budget" REAL, 
	PRIMARY KEY ("CampaignID")
)

/*
3 rows from Campaign tab